In [1]:
def reverse_sub_list(circle_list, pos, length):
    if pos + length <= len(circle_list):
        circle_list[pos: pos + length]  = reversed(circle_list[pos: pos + length])
    else:
        offset = pos + length - len(circle_list) # amount of elements remaining at the end
        sub_list_to_reverse = circle_list[pos:] + circle_list[:offset]
        sub_list_reversed = list(reversed(sub_list_to_reverse))
        circle_list[pos:] = sub_list_reversed[:len(circle_list)-pos]
        circle_list[:offset] = sub_list_reversed[len(circle_list)-pos:] 
        
    return circle_list


def process_circle_list(lengths, repeat):
    circle_list = list(range(256))
    cur_pos = 0
    skip_size = 0
    
    for i in range(repeat):
        for length in lengths:
            circle_list = reverse_sub_list(circle_list, cur_pos, length) 
            cur_pos = (cur_pos + length + skip_size) % len(circle_list)
            skip_size += 1
            
    return circle_list


def build_ascii_list(data):
    data_ascii = []
    for c in data:
        data_ascii.append(ord(c))        
        
    data_ascii += [17, 31, 73, 47, 23] 
    
    return data_ascii


def calc_dense_hashes(sparse_hashes):
    dense_hashes = []
    xor_sum = 0
    
    for count, number in enumerate(sparse_hashes):
        xor_sum = xor_sum ^ number
        if count % 16 == 15:
            dense_hashes.append(xor_sum)
            xor_sum = 0
            
    return dense_hashes


def build_hex_string(dense_hashes):
    hex_string = ''
    
    for dense_hash in dense_hashes:
        # 0:  --> first parameter (dense_hash)
        # 0   --> fill with zeroes
        # {1} --> length of filled string (parameter 2)
        # x   --> hex representation
        hash_as_hex = '{0:0{1}x}'.format(dense_hash, 2)
        hex_string += hash_as_hex
        
    return hex_string


with open('day10_input.txt', 'r') as data:
    data = data.read()
    
    #### Part One
    lengths = list(map(int, data.split(',')))
    
    circle_list = process_circle_list(lengths=lengths, repeat=1)        
    print('Product of first two numbers: {}'.format(circle_list[0]*circle_list[1]))
    
    #### Part Two
    data_ascii = build_ascii_list(data)   
                
    sparse_hashes = process_circle_list(lengths=data_ascii, repeat=64) 
    
    dense_hashes = calc_dense_hashes(sparse_hashes)
                  
    print('Knot hash is: {}'.format(build_hex_string(dense_hashes)))

Product of first two numbers: 62238
Knot hash is: 2b0c9cc0449507a0db3babd57ad9e8d8


In [2]:
from collections import deque
from functools import reduce
from operator import xor


def process_circle_list(lengths, repeat):
    circle_list = deque(range(256))
    cur_pos = deque(range(256))
    skip_size = 0
    
    for _ in range(repeat):
        for length in lengths:
            circle_list.extend(reversed(deque(circle_list.popleft() for _ in range(length))))
            circle_list.rotate(-skip_size)
            cur_pos.rotate(-length-skip_size)
            skip_size += 1
            
    circle_list.rotate(cur_pos.popleft())
            
    return circle_list

            
with open('day10_input.txt', 'r') as data:
    data = data.read()
    
    #### Part One
    lengths = list(map(int, data.split(',')))
    
    circle_list = process_circle_list(lengths=lengths, repeat=1)        
    print('Product of first two numbers: {}'.format(circle_list[0]*circle_list[1]))
    
    #### Part Two
    data_ascii = [ord(c) for c in data]        
    data_ascii.extend([17, 31, 73, 47, 23])  
                
    sparse_hashes = process_circle_list(lengths=data_ascii, repeat=64) 
    sparse_hashes = list(sparse_hashes)
    
    dense_hashes = [reduce(xor, sparse_hashes[i*16:(i+1)*16]) for i in range(16)]
    
    print('Knot hash is: {}'.format(''.join('%02x'%x for x in dense_hashes)))

Product of first two numbers: 62238
Knot hash is: 2b0c9cc0449507a0db3babd57ad9e8d8
